In [ ]:
import uuid

from python_on_whales import DockerClient, docker

docker_clients = []
for i in range(1):
    worker_id = uuid.uuid4().hex
    docker_client = DockerClient(
        compose_project_name=worker_id,
        compose_files=["client-compose.yml"],
    )
    docker_clients.append(docker_client)



In [ ]:
for docker_client in docker_clients:
    docker_client.compose.up(
        detach=True,
        build=True,
    )

In [ ]:
docker.image.list()

In [ ]:
for docker_client in docker_clients:
    docker_client.compose.down()

In [ ]:
list(docker_clients[0].compose.ps()[0].network_settings.networks.values())[0].ip_address

In [ ]:
for docker_client in docker_clients:
    for container in docker_client.compose.ps():
        A_WORKER_ID = docker_client.client_config.compose_project_name
        B_WORKER_ID = [env for env in container.config.env if "WORKER_ID" in env][0].split("=")[1]
        print(A_WORKER_ID, B_WORKER_ID, A_WORKER_ID == B_WORKER_ID, container.name)

In [ ]:
container.name.removeprefix("f4425bc7e176405e89ca8af2954caa46-")

In [ ]:
from httpx import Client

http_client = Client(
    base_url="http://172.31.0.3:11000/apis",
)
http_client.get("/optimizations/evaluates/runs")

In [ ]:
import autocode

optimization = autocode.Optimization(
    num_cpus=2,
    server_host="0.0.0.0",
    server_port=10000,
    dashboard_port=10001,
)

In [ ]:
!python3 -m streamlit run ../autocode/dashboard.py --server.port 10001

In [1]:
from autocode import OptimizationUseCase, ApplicationSetting, OneDatastore
from autocode.container import ApplicationContainer
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "autocode"

application_container: ApplicationContainer = ApplicationContainer()
application_setting: ApplicationSetting = application_container.settings.application()
application_setting.num_cpus = 2
application_setting.server_host = "0.0.0.0"
application_setting.server_port = 10000
application_setting.dashboard_port = 10001
optimization: OptimizationUseCase = application_container.use_cases.optimization()
optimization.reset()

In [2]:
docker_clients = optimization.deploy(
    compose_files=["./client-compose.yml"],
)

#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 DONE 0.0s

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 119B 0.0s done
#1 DONE 0.0s

#2 [app-account internal] load build definition from .Dockerfile
#2 transferring dockerfile: 119B 0.0s done
#2 DONE 0.0s

#3 [app-account internal] load metadata for docker.io/library/golang:latest
#3 ERROR: failed to do request: Head "https://registry-1.docker.io/v2/library/golang/manifests/latest": dial tcp: lookup registry-1.docker.io on 1.0.0.1:53: read udp 172.25.4.192:53732->1.0.0.1:53: i/o timeout
------
 > [app-account internal] load metadata for docker.io/library/golang:latest:
------


failed to solve: golang:latest: failed to resolve source metadata for docker.io/library/golang:latest: failed to do request: Head "https://registry-1.docker.io/v2/library/golang/manifests/latest": dial tcp: lookup registry-1.docker.io on 1.0.0.1:53: read udp 172.25.4.192:53732->1.0.0.1:53: i/o timeout


DockerException: The command executed was `/usr/bin/docker compose --file ./client-compose.yml --project-name 6efe7b677b3449879d77a38330404db1 up --build --detach`.
It returned with code 17
The content of stdout can be found above the stacktrace (it wasn't captured).
The content of stderr can be found above the stacktrace (it wasn't captured).

In [ ]:
from autocode import OptimizationObjective
from typing import Dict, Any, List
from autocode import OptimizationEvaluateRunResponse

objectives: List[OptimizationObjective] = [
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),

    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
]


def evaluator(inputs: List[OptimizationEvaluateRunResponse]) -> Dict[str, Any]:
    output: Dict[str, Any] = {
        "F": inputs[0].objectives + inputs[1].objectives,
    }

    return output


In [ ]:
result = optimization.run(
    objectives=objectives,
    num_inequality_constraints=0,
    num_equality_constraints=0,
    evaluator=evaluator
)

In [ ]:

from sqlmodel import Session, select
from autocode.model import Cache
from typing import List
from autocode.model import OptimizationClient
import dill
import uuid

one_datastore: OneDatastore = application_container.datastores.one()
session: Session = one_datastore.get_session()
client_caches: List[Cache] = list(session.exec(select(Cache).where(Cache.key.startswith("clients"))))
for client_cache in client_caches:
    client: OptimizationClient = dill.loads(client_cache.value)
    print(client.id, client.name, client.host, client.port, client.worker_id, client.is_ready)
    for variable_id, variable in client.variables.items():
        print(variable.get_client_id(), variable_id)

In [ ]:
client: OptimizationClient = OptimizationClient(
    name="app-product",
    host="127.0.0.1",
    port=0,
    worker_id=uuid.uuid4().hex,
    variables={}
)

client_cache: Cache = Cache(
    key=f"clients_{client.id}",
    value=dill.dumps(client),
)

session.add(client_cache)

session.commit()
session.close()